In [52]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import random
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import shutil

sys.path.append('../')
from meta_fusion.benchmarks import *
from meta_fusion.methods import *
from meta_fusion.models import *
from meta_fusion.utils import *
from meta_fusion.third_party import *
from meta_fusion.synthetic_data import Prepare_synthetic_data
from meta_fusion.config import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
# Fixed data parameters
repetition=1
seed=1
scale=0

# Data model parameters
n = 2000
[d1, d2] = dim_modalities = [500, 100]
dim_latent = [20, 20, 0]
noise_ratios = [0.5, 0.1]
trans_type = ["quadratic", "linear", "linear"]
mod_prop = [1, 1, 0, 1]
interactive_prop = 0.1

# mod1_outs = [40, 60, 80, 100]
# mod2_outs = [40, 60, 80, 100]

mod1_outs = [60]
mod2_outs = [40, 60]
combined_hiddens = [300,200,100]
mod1_hiddens = mod2_hiddens = [128]

# data parameters
data_name = 'regression'
exp_name = data_name + "_" + "quadratic_early"
output_dim = 1  # specify the output dimension for regression


extractor_type = 'PCA'
seperate=False
is_static_mod1=True
is_static_mod2=True
freeze_extractor_mod1=False
freeze_extractor_mod2=False

# Load default model configurations 
config = load_config('../experiments_synthetic/config.json')
extractor_config = load_config('../experiments_synthetic/config_extractor.json')

# Model files directory
ckpt_dir = f"./checkpoints/{exp_name}/scale{scale}_seed{seed}/"
config['ckpt_dir'] = extractor_config['ckpt_dir'] = ckpt_dir

# Update other training parameters
config['divergence_weight_scale'] =  scale
config['output_dim'] = extractor_config['output_dim'] = output_dim
config["init_lr"] = 0.001
config["epochs"] = 30
extractor_config["init_lr"] = 0.001

#####################
#    Load Dataset   #
#####################
data_preparer = Prepare_synthetic_data(data_name = data_name, test_size = 0.2, val_size = 0.2)

In [54]:
def run_single_experiment(config, extractor_config, n,
                          random_state, 
                          run_oracle=False, run_coop=True, run_all_at_once=False):

    config['random_state'] = random_state
    extractor_config['random_state'] = random_state
    res_list = []
    best_rho = {}
    cohort_pairs = {}

    #----------------#
    # Split dataset  #
    #----------------#
    train_loader, val_loader, test_loader, oracle_train_loader, oracle_val_loader, oracle_test_loader =\
    data_preparer.get_data_loaders(n, trans_type=trans_type, mod_prop=mod_prop, 
                                   interactive_prop = interactive_prop,
                                   dim_modalities=dim_modalities, dim_latent=dim_latent,
                                   noise_ratios=noise_ratios, random_state=random_state)
    # Get data info
    data_info = data_preparer.get_data_info()
    d1 = data_info[0]
    d2 = data_info[1]
    n = data_info[2]
    n_train = data_info[3]
    n_val = data_info[4]
    n_test = data_info[5]

    print(f"Finished splitting {data_name} dataset. Data information are summarized below:\n"
            f"Modality 1 dimension: {d1}\n"
            f"Modality 2 dimension: {d2}\n"
            f"Data size: {n}\n"
            f"Train size: {n_train}\n"
            f"Val size: {n_val}\n"
            f"Test size: {n_test}")
    sys.stdout.flush() 

    #------------------#
    # Benchmark models #
    #------------------#
    bm_extractor = Extractors([0, d1], [0,d2], d1, d2, train_loader, val_loader)
    _ = bm_extractor.get_dummy_extractors()
    bm_cohort = Cohorts(extractors=bm_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim)

    if run_oracle:
        oracle_d1 = dim_latent[0]
        oracle_d2 = dim_latent[1]+dim_latent[2]
        oracle_extractor = Extractors([0, oracle_d1], [0, oracle_d2], oracle_d1, oracle_d2, oracle_train_loader, oracle_val_loader)
        _ = oracle_extractor.get_dummy_extractors()
        oracle_cohort = Cohorts(extractors=oracle_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim)

    #----------------------------#
    # Proposed model: MetaJoint  #
    #----------------------------#
    meta_extractor = Extractors(mod1_outs, mod2_outs, d1, d2, train_loader, val_loader)
    if extractor_type == 'encoder':
        _ = meta_extractor.get_encoder_extractors(mod1_hiddens, mod2_hiddens, seperate=seperate, config=extractor_config)
    elif extractor_type == 'PCA':
        _ = meta_extractor.get_PCA_extractors()
    meta_cohort = Cohorts(extractors=meta_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim,
                          is_static_mod1=is_static_mod1, is_static_mod2=is_static_mod2,
                          freeze_extractor_mod1=freeze_extractor_mod1, freeze_extractor_mod2=freeze_extractor_mod2)


    #------------------------------#
    #  Train and test benchmarks   #
    #------------------------------#
    # bm_models = bm_cohort.get_cohort_models()
    # _, bm_dims = bm_cohort.get_cohort_info()
    # bm = Benchmarks(config, bm_models, bm_dims, [train_loader, val_loader])
    # bm.train()
    # res = bm.test(test_loader)
    # res_list.append(res)
    # print(f"Finished running basic benchmarks!")
    
    if run_oracle:
        oracle_config = config
        oracle_config["init_lr"] = 0.001
        oracle_models = oracle_cohort.get_cohort_models()
        _, oracle_dims = oracle_cohort.get_cohort_info()
        oracle = Benchmarks(config, oracle_models, oracle_dims, [oracle_train_loader, oracle_val_loader])
        oracle.train()
        res = oracle.test(oracle_test_loader)
        res = {f"oracle_{key}": value for key, value in res.items()}
        res_list.append(res)
        print(f"Finished running oracle benchmarks!")
        
    if run_coop:
        bm_models = bm_cohort.get_cohort_models()
        _, bm_dims = bm_cohort.get_cohort_info()    
        coop = Coop(config, bm_models, bm_dims, [train_loader, val_loader])
        coop.train()
        res = coop.test(test_loader)
        res_list.append(res)
        best_rho['coop'] = coop.best_rho
        print(f"Finished running coop!")

    
    #------------------------------#
    #  Train and test MetaJoint   #
    #------------------------------#
    cohort_models = meta_cohort.get_cohort_models()
    _, dim_pairs = meta_cohort.get_cohort_info()
    metafuse = Trainer(config, cohort_models, [train_loader, val_loader])
    metafuse.train() 
    res = metafuse.test(test_loader)
    res_list.append(res)
    
    metafuse.train_ablation() 
    res = metafuse.test_ablation(test_loader)
    res_list.append(res)
    
    best_rho['meta_learner'] = metafuse.best_rho
    cohort_pairs['cohort'] = dim_pairs
    print(f"Finished running meta fusion!")
    
    
    if run_all_at_once:
        cohort_models = meta_cohort.get_cohort_models()
        config['epochs']=30
        allin1 = Trainer_all_at_once(config, cohort_models, [train_loader, val_loader])
        allin1.train()
        res = allin1.test(test_loader)
        res_list.append(res)
        best_rho['all_at_once'] = allin1.best_rho
        print(f"Finished running meta fusion all in one model!")

    
    results = []
    for res in res_list:
        for method, val in res.items():
            results.append({'Method': method, 'Test_metric': val, 
                            'best_rho':best_rho.get(method), 'cohort_pairs':cohort_pairs.get(method)})
    

    results = pd.DataFrame(results)

    results['random_state']=random_state
    results["d1"] = d1
    results["d2"] = d2
    results['n'] = n
    results['n_train'] = n_train
    results['n_val'] = n_val
    results['n_test'] = n_test 
    results['scale'] = scale

    return results

In [55]:
results = []

#for i in tqdm(range(1, repetition+1), desc="Repetitions", leave=True, position=0):
for i in range(1, repetition+1):
    print(f'Running with repetition {i}...')
    random_state = repetition * (seed-1) + i
    set_random_seed(random_state)
    
    # Run experiment
    tmp = run_single_experiment(config, extractor_config, n, run_oracle=False, run_coop=False, random_state=random_state)
    
    results.append(tmp)

results = pd.concat(results, ignore_index=True)


#####################
#    Save Results   #
#####################
# results.to_csv(outfile, index=False)
# print("\nResults written to {:s}\n".format(outfile))
# sys.stdout.flush()

# After the job is done, remove the model directory to free up space
if os.path.exists(ckpt_dir):
    print(f"Deleting the model checkpoint directory: {ckpt_dir}")
    shutil.rmtree(ckpt_dir)
    print(f"Model checkpoint directory {ckpt_dir} has been deleted.")

Running with repetition 1...
Finished splitting regression dataset. Data information are summarized below:
Modality 1 dimension: 500
Modality 2 dimension: 100
Data size: 2000
Train size: 1280
Val size: 320
Test size: 400
Start training student cohort...
Training with disagreement penalty = 0

Epoch: 1/30 - LR: 0.001000


100%|█████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:02<00:00, 600.94it/s]


model_1: train loss: 408.705, train task loss: 408.705 - val loss: 279.459, val task loss: 279.459 [*] Best so far
model_2: train loss: 440.947, train task loss: 440.947 - val loss: 262.540, val task loss: 262.540 [*] Best so far

Epoch: 2/30 - LR: 0.001000


100%|█████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:02<00:00, 583.37it/s]


model_1: train loss: 218.667, train task loss: 218.667 - val loss: 194.831, val task loss: 194.831 [*] Best so far
model_2: train loss: 232.896, train task loss: 232.896 - val loss: 205.024, val task loss: 205.024 [*] Best so far

Epoch: 3/30 - LR: 0.001000


...

1280it [00:01, 1066.47it/s]                                                                                            


meta_learner: train task loss: 357.224 - val task loss: 297.993 [*] Best so far


1280it [00:01, 1069.67it/s]                                                                                            


meta_learner: train task loss: 316.696 - val task loss: 268.519 [*] Best so far
Done!
Method: (simple_average), Test_MSE: 57.140743255615234
Method: (weighted_average), Test_MSE: 57.05751037597656
Method: (meta_learner), Test_MSE: 305.0787353515625
Method: (best_single), Test_MSE: 60.327919006347656
Method: (cohort), Test_MSE: [tensor(64.2954), tensor(60.3279)]
Finished running meta fusion!
Deleting the model checkpoint directory: ./checkpoints/regression_quadratic_early/scale0_seed1/


NameError: name 'shutil' is not defined

In [ ]:
# Convert tensor values in 'Test_MSE' column to float
def convert_to_float(value):
    if isinstance(value, torch.Tensor):
        return value.item()  # Convert tensor to float
    elif isinstance(value, list):
        return [v.item() if isinstance(v, torch.Tensor) else v for v in value]  # Convert list of tensors
    return value

In [ ]:
import matplotlib.pyplot as plt
results_pd['Test_metric'] = results_pd['Test_metric'].apply(convert_to_float)

In [ ]:
# Filter out methods you don't want to include
methods_to_include = ['oracle_mod1', 'oracle_mod2', 'oracle_early_fusion', 'oracle_late_fusion', 
                      'mod1', 'mod2', 'early_fusion', 'late_fusion',  
                      'meta_learner', 'coop']  # Change this list based on your needs
df_filtered = results_pd[results_pd['Method'].isin(methods_to_include)]

In [ ]:
# Mean and standard deviation of Test_MSE for each method
grouped_stats = df_filtered.groupby('Method')['Test_metric'].agg(['mean', 'std'])

print(grouped_stats)

In [ ]:
results_pd